In [37]:
from sklearn.svm import SVC
from sklearn.datasets import make_classification
import numpy as np
import os
import glob
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold

In [48]:
print(os.getcwd())
current = os.getcwd()
data_folder = os.path.join(current, "normalized_images_128")
chess_types_folders = glob.glob(os.path.join(data_folder, "*"))
# print(chess_types_folders)
pieces_info = []
labels = {"King": 1, "Knight":2, "Bishop":3, "Rook":4, "Pawn":5, "Queen":6}
for chess_types in chess_types_folders:
    pieces = glob.glob(f'{chess_types}/*')
    # print(pieces)
    type = chess_types.split("/")[-1]
    for piece in pieces:
        p = {"normalized_img": np.load(piece).reshape(-1), "label": labels[type]}
        pieces_info.append(p)
chess_df = pd.DataFrame(pieces_info)

/Users/giangto/Documents/umn/csci5525/5525Chess


In [ ]:
#sample df test set and df train set
chess_test = pd.DataFrame()
for i in range(1,7):
    chess_i = chess_df[chess_df["label"]==i].sample(frac=0.3)
    chess_test = pd.concat([chess_i,chess_test])
chess_test = chess_test[chess_df.columns]
print(chess_test.head())
chess_df = chess_df.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)
chess_test = chess_test.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)

# Merge
chess_train = chess_df.merge(chess_test, how='left', indicator=True)
chess_train = chess_train[chess_train['_merge'] == 'left_only'].drop(columns='_merge')

#mini dataset
# chess_train = chess_train.sample(frac=0.5)
# chess_test = chess_test.sample(frac=0.5)

                                        normalized_img  label
758  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...      6
762  [0.8117647058823529, 0.8156862745098039, 0.819...      6
816  [0.45098039215686275, 0.47058823529411764, 0.4...      6
787  [0.5882352941176471, 0.592156862745098, 0.5921...      6
807  [0.26666666666666666, 0.25882352941176473, 0.2...      6


/var/folders/tp/pqhy6s2j2lq2h7_jn7jz7gpr0000gn/T/ipykernel_821/949022291.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  chess_df = chess_df.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)
/var/folders/tp/pqhy6s2j2lq2h7_jn7jz7gpr0000gn/T/ipykernel_821/949022291.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  chess_test = chess_test.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)


In [50]:
print("Chess train label counts:")
print(chess_train['label'].value_counts())
print()
print("Chess test label counts:")
print(chess_test['label'].value_counts())

print(chess_train.shape)
print(chess_test.shape)

Chess train label counts:
label
5    109
2    109
4    106
3     96
6     90
1     88
Name: count, dtype: int64

Chess test label counts:
label
5    47
2    47
4    46
3    41
6    38
1    38
Name: count, dtype: int64
(598, 2)
(257, 2)


In [51]:
#Load dataset
X_train = np.array(chess_train['normalized_img'])
X_train = np.array([x for x in X_train])
y_train = np.array(chess_train['label'])

X_test = np.array(chess_test['normalized_img'])
X_test = np.array([x for x in X_test])
y_test = np.array(chess_test['label'])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)

X_train shape:  (598, 16384)
X_test shape:  (257, 16384)


In [52]:
#implement own version of SVM and tweak something?????
#DIY SVM
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances

# Define the trigonometric kernel function
class SVM:
    def __init__(self, kernel='linear', C=10000.0, max_iter=100000, degree=3, gamma=1, sigma=1):
        self.kernel = {
        'poly': lambda x, y: np.dot(x, y.T) ** degree,
        'rbf': lambda x, y: np.exp(-gamma * np.sum((x[:, np.newaxis] - y) ** 2, axis=2)),
        'linear': lambda x, y: np.dot(x, y.T),
        'string': lambda x, y: np.sum(x[:, np.newaxis] == y, axis=2).astype(float),
        'histogram_intersection': lambda x, y: np.sum(np.minimum(x[:, np.newaxis], y), axis=2),
        'chi_square': lambda x, y: 1 - 0.5 * np.sum(((x[:, np.newaxis] - y) ** 2) / (x[:, np.newaxis] + y + 1e-10), axis=2),
        'laplacian': lambda x, y: np.exp(-np.sum(np.abs(x[:, np.newaxis] - y), axis=2) / sigma),
        'random_walk': lambda x, y: random_walk_kernel_placeholder(x, y),
        }[kernel]
        self.C = C
        self.max_iter = max_iter
    
    def random_walk_kernel_placeholder(x, y):
        """
        Placeholder for a random walk kernel. Random walk kernels require graph representations.
        This function should be replaced with an actual implementation.
        """
        # Example: Return a dummy similarity matrix for demonstration purposes
        return np.dot(x, y.T)  # Replace with graph-based computations

    def restrict_to_square(self, t, v0, u):
        t = (np.clip(v0 + t * u, 0, self.C) - v0)[1] / u[1]
        return (np.clip(v0 + t * u, 0, self.C) - v0)[0] / u[0]

    def fit(self, X, y):
        self.X = X.copy()
        self.y = y * 2 - 1  # Convert labels to {-1, 1}
        self.lambdas = np.zeros_like(self.y, dtype=float)
        self.K = self.kernel(self.X, self.X)

        # print("Kernel matrix shape:", self.K.shape)
        # print("Kernel matrix example:", self.K[:5, :5])


        for _ in range(self.max_iter):
            for idxM in range(len(self.lambdas)):
                idxL = np.random.randint(0, len(self.lambdas))
                Q = self.K[[[idxM, idxM], [idxL, idxL]], [[idxM, idxL], [idxM, idxL]]]
                v0 = self.lambdas[[idxM, idxL]]
                k0 = 1 - np.sum(self.lambdas * self.K[[idxM, idxL]], axis=1)
                u = np.array([-self.y[idxL], self.y[idxM]])
                t_max = np.dot(k0, u) / (np.dot(np.dot(Q, u), u) + 1E-15)
                self.lambdas[[idxM, idxL]] = v0 + u * self.restrict_to_square(t_max, v0, u)

        idx = np.nonzero(self.lambdas > 1E-15)[0]  # Non-zero support vector indices
        if len(idx) > 0:
            self.b = np.sum((1.0 - np.sum(self.K[idx] * self.lambdas, axis=1)) * self.y[idx]) / len(idx)
        else:
            self.b = 0  # Default bias if no support vectors

    def decision_function(self, X):
        return np.sum(self.kernel(X, self.X) * self.y * self.lambdas, axis=1) + self.b

    def predict(self, X):
        decision_values = self.decision_function(X)
        return np.sign(decision_values)


class MultiClassSVM:
    def __init__(self, base_svm_class, n_classes, **svm_params):
        self.n_classes = n_classes
        self.models = [base_svm_class(**svm_params) for _ in range(n_classes)]  # One SVM per class

    def fit(self, X, y):
        """
        Train one binary SVM for each class (One-vs-Rest strategy).
        """
        self.classes_ = np.unique(y)  # Unique class labels
        for i, cls in enumerate(self.classes_):
            binary_y = np.where(y == cls, 1, -1)  # Convert to binary labels: current class vs others
            print(f"Training SVM for class {cls} vs rest...")
            self.models[i].fit(X, binary_y)

    def predict(self, X):
        """
        Predict the class for each input sample.
        """
        decision_values = np.array([model.decision_function(X) for model in self.models])
        normalized_scores = (decision_values - decision_values.min(axis=0)) / (decision_values.max(axis=0) - decision_values.min(axis=0))
        predicted_classes = self.classes_[np.argmax(normalized_scores, axis=0)]

        return predicted_classes

# RBF kernel

In [29]:
#gamma = X_train.shape[1]
svm = MultiClassSVM(SVM, n_classes=6, kernel='rbf', C=10, max_iter=10000, gamma=0.0001)

# Train the multi-class SVM
svm.fit(X_train, y_train)

# Predict and evaluate
y_test_pred = svm.predict(X_test)
print("Classification Report:")
print("ytest: ", y_test)
print("ypred: ", y_test_pred) 
print(classification_report(y_test, y_test_pred))

final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"RBF SVM Performance Metrics with C=1.0, gamma='scale':")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Training SVM for class 1 vs rest...
Training SVM for class 2 vs rest...
Training SVM for class 3 vs rest...
Training SVM for class 4 vs rest...
Training SVM for class 5 vs rest...
Training SVM for class 6 vs rest...
Classification Report:
ytest:  [2 4 5 4 1 6 1 3 6 4 4 4 6 3 3 1 6 6 6 1 4 3 3 2 5 1 4 2 3 5 4 5 4 6 4 2 6
 2 6 3 3 2 1 6 5 3 3 6 6 4 4 4 5 2 2 2 4 4 1 3 2 4 1 2 4 5 3 2 1 4 2 5 5 2
 1 3 1 6 4 1 4 5 6 4 5 4 5 2 2 6 2 4 4 2 4 5 6 3 3 1 4 3 1 6 3 2 1 5 4 2 2
 1 3 3 5 6 2 1 5 5 3 1 6 5 5 6 6 5]
ypred:  [5 5 5 5 5 5 5 3 5 6 5 5 6 4 2 5 5 6 6 5 5 6 5 3 6 4 6 5 5 5 5 3 5 6 3 5 6
 5 6 6 5 5 5 5 6 5 6 5 5 4 4 6 6 5 6 5 4 5 5 5 5 4 5 6 5 5 5 6 5 5 5 6 2 6
 3 6 5 6 5 5 5 5 5 2 5 3 5 6 2 6 5 6 5 5 5 5 5 6 6 5 5 5 5 6 6 5 4 5 5 5 5
 5 2 5 5 5 5 6 2 6 6 5 2 5 5 6 6 2]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        18
           2       0.12      0.05      0.07        22
           3       0.17      0.05      0.08        20
      

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

# RBF with pca and pipeline

In [55]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Step 1: Standardize the features
    ('pca', PCA(n_components=250)), 
    ('svm', MultiClassSVM(SVM, n_classes=6, kernel='rbf', C=10, max_iter=10000, gamma=0.001))  # Step 2: Train the Multi-Class SVM
])

# Train the pipeline
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_test_pred = pipeline.predict(X_test)

# Classification metrics
print("Classification Report:")
print("ytest: ", y_test)
print("ypred: ", y_test_pred)
print(classification_report(y_test, y_test_pred))

final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

# Print results
print(f"Final Test Error: {final_test_error:.4f}")
print(f"Multi-Class SVM Performance Metrics with Laplacian Kernel:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))


Training SVM for class 1 vs rest...
Training SVM for class 2 vs rest...
Training SVM for class 3 vs rest...
Training SVM for class 4 vs rest...
Training SVM for class 5 vs rest...
Training SVM for class 6 vs rest...
Classification Report:
ytest:  [6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
ypred:  [5 5 5 5 5 1 5 5 3 4 5 4 5 5 4 1 4 5 5 5 5 5 5 5 5 5 6 1 4 5 5 4 4 5 5 4 4
 5 5 4 5 5 5 5 5 4 5 5 5 5 4 5 5 4 4 4 5 5 5 5 3 5 5 5 2 5 5 5 5 5 3 5 5 5
 5 5 5 4 6 5 5 5 5 5 6 5 4 5 5 4 5 5 5 5 5 5 5 5 5 5 3 2 5 4 4 5 5 2 5 5 

In [ ]:
# Laplacian

In [47]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Step 1: Standardize the features
    ('pca', PCA(n_components=250)), 
    ('svm', MultiClassSVM(SVM, n_classes=6, kernel='laplacian', C=10, max_iter=10000, gamma=0.001))  # Step 2: Train the Multi-Class SVM
])

# Train the pipeline
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_test_pred = pipeline.predict(X_test)

# Predict and evaluate
y_test_pred = svm.predict(X_test)
print("Classification Report:")
print("ytest: ", y_test)
print("ypred: ", y_test_pred) 
print(classification_report(y_test, y_test_pred))

final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"RBF SVM Performance Metrics with C=1.0, gamma='scale':")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Training SVM for class 1 vs rest...
Training SVM for class 2 vs rest...
Training SVM for class 3 vs rest...
Training SVM for class 4 vs rest...
Training SVM for class 5 vs rest...
Training SVM for class 6 vs rest...
Classification Report:
ytest:  [2 4 5 4 1 6 1 3 6 4 4 4 6 3 3 1 6 6 6 1 4 3 3 2 5 1 4 2 3 5 4 5 4 6 4 2 6
 2 6 3 3 2 1 6 5 3 3 6 6 4 4 4 5 2 2 2 4 4 1 3 2 4 1 2 4 5 3 2 1 4 2 5 5 2
 1 3 1 6 4 1 4 5 6 4 5 4 5 2 2 6 2 4 4 2 4 5 6 3 3 1 4 3 1 6 3 2 1 5 4 2 2
 1 3 3 5 6 2 1 5 5 3 1 6 5 5 6 6 5]
ypred:  [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        18
           2       0.00      0.00      0.00        22
           3       0.00      0.00      0.00        20
      

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  